# Configuração do ambiente

In [ ]:
pip install cryptography asyncio

### Iremos utilizar o asyncio e asyncio.StreamReader/StreamWriter para criar o servidor (Receiver) e o cliente (Emitter):

    - O servidor escuta conexões e recebe mensagens criptografadas.
    - O cliente envia mensagens criptografadas.
    - A comunicação precisa de um mecanismo de cifra AEAD com SHAKE-256 e chaves derivadas via KDF.

In [ ]:
import subprocess
import time
import threading

# Função para ler e exibir a saída de um processo
def print_output(process, name):
    while True:
        # Lê a saída (stdout e stderr)
        output = process.stdout.readline()
        error = process.stderr.readline()
        
        # Verifica se o processo terminou
        if output == b'' and error == b'' and process.poll() is not None:
            break
        
        # Exibe a saída
        if output:
            print(f"[{name}] {output.strip().decode('utf-8')}")
        if error:
            print(f"[{name} - ERRO] {error.strip().decode('utf-8')}")
        
        time.sleep(0.1)

# Executa o Servidor.py em segundo plano (com buffering desativado)
server_process = subprocess.Popen(
    ["python", "-u", "Servidor.py"],  # -u desativa o buffering
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Executa o Cliente.py em segundo plano (com buffering desativado)
client_process = subprocess.Popen(
    ["python", "-u", "Cliente.py"],  # -u desativa o buffering
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Cria threads para exibir a saída dos processos
server_thread = threading.Thread(target=print_output, args=(server_process, "Servidor"))
client_thread = threading.Thread(target=print_output, args=(client_process, "Cliente"))

# Inicia as threads
server_thread.start()
client_thread.start()

# Termina os processos após 5 segundos
server_process.terminate()
client_process.terminate()

# Aguarda as threads terminarem (opcional)
server_thread.join(timeout=1)  # Timeout de 1 segundo para evitar bloqueio
client_thread.join(timeout=1)  # Timeout de 1 segundo para evitar bloqueio

print("Execução terminada após 5 segundos.")

In [ ]:
import asyncio
from crypto_utils import decrypt_message

key_receiver = b'minha_chave'

async def handle_client(reader, writer):
    """Recebe mensagens criptografadas e decifra-as"""
    print("Cliente conectado!")
    try:
        while True:
            data = await reader.read(1024)
            if not data:
                print("Cliente desconectado.")
                break

            nonce, ciphertext, tag = data[:16], data[16:-16], data[-16:]
            print(f"Defini nonce: {nonce}, ciphertext: {ciphertext}, tag {tag}")

            message = decrypt_message(key_receiver, nonce, ciphertext, tag)
            print(f"Mensagem Recebida e decifrada: {message.decode()}")

    except Exception as e:
        print(e)
    finally:
        writer.close()
        await writer.wait_closed()
        print("Conexão com o cliente encerrada.")

async def start_server():
    """Inicia o servidor"""
    print("Servidor iniciado!")
    server = await asyncio.start_server(handle_client, '127.0.0.1', 8000)
    async with server:
        await server.serve_forever()

print("Iniciando servidor...")
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    print("Loop de eventos já em execução. Criando uma tarefa para o servidor...")
    task = asyncio.create_task(start_server())
else:
    print("Iniciando novo loop de eventos...")
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.create_task(start_server())

Iniciando servidor...
Loop de eventos já em execução. Criando uma tarefa para o servidor...


Servidor iniciado!
Cliente conectado!
Defini nonce: b'\xd1\xf2W\xa2\xae\xecy\t\xc7\x074\x8c\x1d\xb7\xa5\x9a', ciphertext: b"\x11'~", tag b'\xf1Nn\xadf\xf5\x06\xf9Q\xd1\xd6z\xf8`w\xe1'
Mensagem Recebida e decifrada: Ola
Cliente desconectado.
Conexão com o cliente encerrada.


In [2]:
import asyncio
from crypto_utils import encrypt_message

# Defina a chave do emissor (deve ser a mesma usada no servidor)
key_emitter = b'minha_chave'

async def send_message(writer):
    """Envia uma mensagem criptografada para o servidor"""
    message = "Ola"
    
    # Criptografa a mensagem (a mensagem já é codificada em bytes antes de ser passada para a função)
    nonce, ciphertext, tag = encrypt_message(key_emitter, message)

    writer.write(nonce + ciphertext + tag)
    await writer.drain()

    print("Mensagem Enviada!")
    writer.close()

async def start_client():
    """Cliente que envia mensagens criptografadas"""
    print("Cliente inicializado!")
    reader, writer = await asyncio.open_connection('127.0.0.1', 8000)
    print("Cliente conectado")
    print("     reader: ", reader)
    print("     writer: ", writer)

    await send_message(writer)

# Executa o cliente
print("Iniciando cliente...")
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    print("Loop de eventos já em execução. Criando uma tarefa para o cliente...")
    task = asyncio.create_task(start_client())
else:
    print("Iniciando novo loop de eventos para o cliente...")
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.create_task(start_client())

Iniciando cliente...
Loop de eventos já em execução. Criando uma tarefa para o cliente...


Cliente inicializado!
Cliente conectado
     reader:  <StreamReader transport=<_SelectorSocketTransport fd=1528 read=polling write=<idle, bufsize=0>>>
     writer:  <StreamWriter transport=<_SelectorSocketTransport fd=1528 read=polling write=<idle, bufsize=0>> reader=<StreamReader transport=<_SelectorSocketTransport fd=1528 read=polling write=<idle, bufsize=0>>>>
Mensagem Enviada!
